<h1>Cleaning datasets</h1>

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

<h2>Dataset "Bruker"</h2>

I could easily figure out how to load the data correctly into a pandas dataframe by looking at the json file and comparing with the pandas read_json() documentation. 

Loading dataset and having an initial look at it:

In [2]:
df_bruker = pd.read_json('C:\\Users\sever\OneDrive\Documents\kode\scripts\INF161\data/project/bruker.json', orient='split')
df_bruker.head()

,BrukerID,Kjonn,Alder,Jobb,Postkode
0,0,None,45.0,6.0,92103
1,1,M,50.0,16.0,55405-2546
2,2,M,18.0,20.0,44089
3,3,M,NaN,1.0,33304
4,4,M,35.0,6.0,48105


In [3]:
round(df_bruker.describe(),2)

,BrukerID,Alder,Jobb
count,6040.00,5046.00,5447.00
mean,3020.47,30.67,9.10
std,1743.80,12.95,11.24
min,0.00,1.00,0.00
25%,1510.75,25.00,3.00
50%,3020.50,25.00,7.00
75%,4530.25,35.00,14.00
max,6040.00,56.00,99.00


Check for duplicate user IDs:

In [4]:
df_bruker['BrukerID'].drop_duplicates().shape[0] == df_bruker['BrukerID'].shape[0]

True

The number of distinct rows in the 'BrukerID' column is the same as number of rows in the full column. Thus I can conclude that the all of the BrukerIDs are indeed unique.

Investigate possible values for gender:

In [5]:
df_bruker['Kjonn'].drop_duplicates()

0    None
1       M
7       F
Name: Kjonn, dtype: object

In [6]:
df_bruker['Kjonn'].value_counts()

M    4126
F    1611
Name: Kjonn, dtype: int64

Investigate possible values for age:

In [7]:
df_bruker['Alder'].value_counts()

25.0    1733
35.0     997
18.0     924
45.0     463
50.0     420
56.0     319
1.0      190
Name: Alder, dtype: int64

Investigating the values in the 'Postkode' column:

In [8]:
df_bruker.loc[df_bruker['Postkode'].str.len() > 5].sort_values(by=['Postkode'], ascending=False)

,BrukerID,Kjonn,Alder,Jobb,Postkode
1361,1362,M,45.0,16.0,98107-2117
270,271,M,NaN,10.0,98043-3621
1079,1080,M,35.0,5.0,96707-1321
880,881,M,35.0,0.0,96707-1321
647,648,F,50.0,3.0,95451-9554
...,...,...,...,...,...
4138,4139,M,NaN,1.0,08854-3115
5555,5556,F,35.0,3.0,02725-1010
293,294,F,NaN,13.0,02136-1522
1137,1138,M,45.0,NaN,01904-1355


The 'Postkode' contains possibly different formats. Some domain knowledge about postal codes could be useful here to determine if some kind of cleanup of this column is necessary (for example by taking just the 5 first numbers of each row). As the different formats are also present in the clean dataset of the first part of the project, I'll just leave them as is for now.

Checking how many rows of each column contains missing values:

In [9]:
df_bruker.isna().sum()

BrukerID      0
Kjonn       303
Alder       994
Jobb        593
Postkode    451
dtype: int64

Three of the columns with missing data are nominal categorical data, and the last one is ordinal categorical data. If you are hellbent on completing these rows, you could apply multivariate imputation and look for correlations between the columns. For now I will not use these features in my machine learning models anyways, so I'll just simply replace the missing values with the most common value for each feature. Before directly using the imputer, I conform the missing value to np.nan.

In [10]:
df_bruker = df_bruker.fillna(np.nan)
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df_bruker_cleaned = pd.DataFrame(imputer.fit_transform(df_bruker), columns=df_bruker.columns)
df_bruker_cleaned.head()

,BrukerID,Kjonn,Alder,Jobb,Postkode
0,0,M,45,6,92103
1,1,M,50,16,55405-2546
2,2,M,18,20,44089
3,3,M,25,1,33304
4,4,M,35,6,48105


Writing to disk in csv-format:

In [11]:
df_bruker_cleaned.to_csv(
    path_or_buf='C:\\Users\sever\OneDrive\Documents\kode\scripts\INF161\data/project/cleaned/bruker.csv',
    index=False)

<h2>Dataset "Film"</h2>

Loading dataset and having an initial look at it:

In [12]:
df_excel = pd.read_excel('C:\\Users\sever\OneDrive\Documents\kode\scripts\INF161\data/project/film.xlsx', sheet_name=None)
df_film = df_excel['film']
df_film.head()

,Unnamed: 0,FilmID,Tittel,Sjanger
0,0,0,Autumn in New York (2000),Drama|Romance
1,1,1,"Vie est belle, La (Life is Rosey) (1987)",Comedy|Drama
2,2,2,Defying Gravity (1997),Drama
3,3,3,Ruthless People (1986),Comedy
4,4,4,Portraits Chinois (1996),Drama


In [13]:
round(df_film.describe(),2)

,Unnamed: 0,FilmID
count,3883.00,3883.00
mean,1941.00,1973.69
std,1121.07,1142.11
min,0.00,0.00
25%,970.50,985.50
50%,1941.00,1973.00
75%,2911.50,2963.50
max,3882.00,3952.00


Checking for missing values:

In [14]:
df_film.isna().sum()

Unnamed: 0    0
FilmID        0
Tittel        0
Sjanger       0
dtype: int64

Investigating the values in the 'Sjanger' column. I want to be able to see a bit more than 10 rows/columns, so I change the display.max_rows/columns setting.

In [15]:
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)

In [16]:
df_film['Sjanger'].value_counts().sort_values(ascending=False).head(20)

Drama                   843
Comedy                  521
Horror                  178
Comedy|Drama            162
Comedy|Romance          142
Drama|Romance           134
Documentary             116
Thriller                101
Action                   65
Drama|Thriller           63
Action|Thriller          48
Crime|Drama              44
Drama|War                43
Romance                  39
Action|Drama             39
Children's|Comedy        34
Comedy|Drama|Romance     34
Western                  33
Horror|Sci-Fi            33
Horror|Thriller          32
Name: Sjanger, dtype: int64

I notice that the movies may have several genres associated with them. In these cases the genres are separated by a '|'. I want to have a look at how many genres a movie can have, could be useful information when I'm going to convert the column to a dummy variable format. 

In [17]:
sjanger_counts = df_film['Sjanger'].str.count('\|').sort_values(ascending=False)
sjanger_counts

3174    5
3218    4
1622    4
1401    4
2446    4
       ..
1583    0
2988    0
2986    0
2985    0
3882    0
Name: Sjanger, Length: 3883, dtype: int64

Seems 6 is the most genres a single movie has in the dataset. Time to do conversion:

First I split the 'Sjanger' column on the '|' separator and expand it into dummy variable columns. I then create a copy of this dataframe.

In [18]:
dfx = df_film['Sjanger'].str.split('|', expand=True)
dfx = pd.get_dummies(dfx)
dfx_2 = dfx.copy()

For each genre combo, I will get a duplicate genre column. I want to merge these into one column for each genre, while keeping the information in the duplicate columns. To do this, I loop through the columns and for each column I loop through again to find other columns with identical suffixes (the duplicates will have names of the type "2_genre"). By only comparing the 5 first character of the genre names, I can also combine genres like "Children's" and "Children" into one. If I find a match, I will store the sum of columns as a new common column for this genre.

In [19]:
for col_1 in dfx.columns:
    result = dfx[col_1].astype(int)    
    genre_1 = col_1.split('_')[1]
    for col_2 in dfx.columns:        
        genre_2 = col_2.split('_')[1]
        if genre_1[:5] == genre_2[:5]:
            result = result+dfx[col_2].astype(int)    
    dfx_2[genre_1] = result

As the loop visited both "Children's" and "Children" I combine them here.

Right now, the new columns will have 0 for the movies where the genre is not represented, and some number > 1 for each row where the genre was represented. To convert this to the standard dummy variable format, I change all positive values to a 1. I can now drop the superfluous genre columns.

In [20]:
dfx_2["Children's"] = dfx_2["Children"]+dfx_2["Children's"]    
dfx_2 = dfx_2.where(dfx_2 == 0, 1)
dfx_2 = dfx_2.drop(dfx_2.columns[:74], axis=1).drop(['Children'], axis=1)

Finally, I will combine the cleaned genre columns with the rest of the columns from the original dataframe. I drop the duplicate index column and old 'Sjanger' column.

I notice a genre called "Ukjennt" which is present only in 3 rows, so I just drop this column too.

In [21]:
df_film_cleaned = pd.concat([df_film, dfx_2], axis=1).drop(['Unnamed: 0', 'Sjanger'], axis=1)
df_film_cleaned = df_film_cleaned.loc[df_film_cleaned['Ukjennt'] != 1].drop(['Ukjennt'], axis=1)

df_film_cleaned.head()

,FilmID,Tittel,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,Autumn in New York (2000),0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,1,"Vie est belle, La (Life is Rosey) (1987)",0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,2,Defying Gravity (1997),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,3,Ruthless People (1986),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,Portraits Chinois (1996),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


The dataset is now cleaned and converted to the correct format, so I can write it to disk as a csv file.

In [22]:
df_film_cleaned.to_csv(
    path_or_buf='C:\\Users\sever\OneDrive\Documents\kode\scripts\INF161\data/project/cleaned/film.csv',
    index=False)

<h2>Dataset "Rangering"</h2>

Loading dataset and having an initial look at it:

In [23]:
df_rang = pd.read_csv('C:\\Users\sever\OneDrive\Documents\kode\scripts\INF161\data/project/rangering.dat', sep='::', header=None)
df_rang.columns = ['BrukerID', 'FilmID', 'Rangering', 'Timestamp']
df_rang.head()

<ipython-input-23-6804d5f2f3ce>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_rang = pd.read_csv('C:\\Users\sever\OneDrive\Documents\kode\scripts\INF161\data/project/rangering.dat', sep='::', header=None)


,BrukerID,FilmID,Rangering,Timestamp
0,0,616,4,959441640.0
1,0,1561,7,959441640.0
2,0,1540,6,959441640.0
3,0,88,5,959441640.0
4,0,620,8,959441640.0


In [24]:
df_rang.describe()

,BrukerID,FilmID,Rangering,Timestamp
count,900188.000000,900188.000000,900188.000000,8.986960e+05
mean,2991.861171,1989.674352,4.279477,9.722414e+08
std,1736.206736,1126.366837,1.971074,1.214672e+07
min,0.000000,0.000000,1.000000,9.567039e+08
25%,1458.000000,1037.000000,3.000000,9.653029e+08
50%,2967.000000,1959.000000,4.000000,9.729904e+08
75%,4501.000000,2963.000000,5.000000,9.752202e+08
max,6040.000000,3952.000000,10.000000,1.046455e+09


Checking for missing values:

In [25]:
df_rang.isna().sum()

BrukerID        0
FilmID          0
Rangering       0
Timestamp    1492
dtype: int64

1492 rows lack a timestamp. As I don't know which scale these values are on (pre/post 2000-08-01), I'll just drop these rows.

In [26]:
df_rang = df_rang.dropna()

Check that all values are between 1 and 10:

In [27]:
df_rang['Rangering'].drop_duplicates().value_counts().sort_values(ascending=False)

1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
Name: Rangering, dtype: int64

Ratings from before 2000-08-01 are on a scale of 1 to 10 while ratings after are on a scale from 1 to 5. We can convert the pre-change ratings to match the post-change ratings by halving the pre-change ratings.

Converting date to unix time:

In [28]:
change_time = (pd.Timestamp('2000-08-01 00:00:00') - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [29]:
pd.Timestamp(965088000, unit='s')

Timestamp('2000-08-01 00:00:00')

Enforcing correct datatypes:

In [30]:
df_rang['Timestamp'] = df_rang['Timestamp'].astype('int64')

In [31]:
df_rang['Rangering'] = df_rang['Rangering'].astype('float64')

In [32]:
df_rang.dtypes

BrukerID       int64
FilmID         int64
Rangering    float64
Timestamp      int64
dtype: object

Separating pre and post change ratings, halving the pre change ratings and finally combining them again

In [33]:
df_before_change = df_rang.loc[df_rang['Timestamp'] < change_time]
df_before_change_halved = df_before_change.copy()
df_before_change_halved['Rangering'] = df_before_change['Rangering']/2

df_after_change = df_rang.loc[df_rang['Timestamp'] >= change_time]

df_rang_cleaned = pd.concat([df_before_change_halved, df_after_change], axis=0)
df_rang_cleaned

,BrukerID,FilmID,Rangering,Timestamp
0,0,616,2.0,959441640
1,0,1561,3.5,959441640
2,0,1540,3.0,959441640
3,0,88,2.5,959441640
4,0,620,4.0,959441640
...,...,...,...,...
900183,6040,1153,4.0,976584194
900184,6040,3714,4.0,976584260
900185,6040,2834,5.0,976584260
900186,6040,48,5.0,976584300


Dataset seems good to go, and I write it to disk as csv.

In [34]:
df_rang_cleaned.to_csv(
    path_or_buf='C:\\Users\sever\OneDrive\Documents\kode\scripts\INF161\data/project/cleaned/rangering.csv',
    index=False)